# Solving a Bridge and Torch Problem using Integer Linear Programming (ILP)

# Introduciton

Suppose 4 individuals need to cross a bridge. Its dark though, so they need to take a lantern with them in order to cross. Additionally, the bridge can only support up to 2 people crossing at a time. How should the 4 individuals move across the bridge such that the total time it takes for everyone to the end is finished?

## More Information

For more information about this problem, feel free to check out the Wikipedia page: https://en.wikipedia.org/wiki/Bridge_and_torch_problem.

# Imports

In [ ]:
from docplex.mp.model import Model
import pandas as pd

# Input Data

## Sets

$i \in I$: The set of people crossing the bridge

In [ ]:
I = ['A', 'B', 'C', 'D']

$t \in T$: The set of time periods



The number of time periods for everyone to cross the bridge:
* 1 Person: 1
* 2 People: 2
* 3 People: 3
* 4 People: 5
* 5 People: 7
* 6 People: 9
* 7 People: 11

Excluding the uninteresting cases of having 2 or fewer people, the number time periods we need is:

$2*len(I)-2$ 

where $len(I)$ is the number of people that need to cross

In [ ]:
T = [x for x in range(2*len(I)-2)]

## Parameters

$c_{i}$: The time it takes person $i \in I$ to cross the bridge

In [ ]:
c = {
    'A': 1,
    'B': 2,
    'C': 5,
    'D': 8
}

# Optimization Model 

## Create a CPLEX model object

In [ ]:
# create a model object
bridge_model = Model(name='bridge+torch')

## Decision Variables

$x_{it}$: $1$ if person $i \in I$ crosses the bridge at time $t \in \{1, ..., |T|\}$ and $0$ otherwise 

In [ ]:
x = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T[1:]], name="x")

$s_{it}$: $1$ if person $i \in I$ is at the start of the bridge at time $t \in T$ and $0$ otherwise 

In [ ]:
s = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T], name="s")

$f_{it}$: $1$ if person $i \in I$ is at the end (finish line) of the bridge at time $t \in T$ and $0$ otherwise 

In [ ]:
f = bridge_model.binary_var_dict(keys = [(i, t) for i in I for t in T], name="f")

$w_{t}$: The time spent traveling across the bridge at $t \in \{1,..|T|\}$ 

In [ ]:
w = bridge_model.continuous_var_dict(keys = [t for t in T], name="w")

## Objective Function

Minimize total walking time:

$minimize \quad \sum_{t = 1}^{|T|} w_{t}$

In [ ]:
total_walk_time = bridge_model.sum(w[t] for t in T[1:])
bridge_model.minimize(total_walk_time)

## Constraints

Everyone starts at the beginning

$s_{i0} = 1, \quad \forall i \in I$

In [ ]:
bridge_model.add_constraints(s[(i,0)] == 1 for i in I)

Everyone ends at the end of the bridge/finish line

$f_{i|T|} = 1, \quad \forall i \in I$



In [ ]:
bridge_model.add_constraints(f[(i,T[-1])] == 1 for i in I)

Odd time periods always result in 2 people crossing

$\sum_{i \in I}x_{it} = 2, \quad \forall$ odd $t \in T$

In [ ]:
bridge_model.add_constraints(bridge_model.sum(x[(i,t)] for i in I) == 2 for t in T if t % 2 == 1)

Even time periods always result in 1 person crossing

$\sum_{i \in I}x_{it} = 1, \quad \forall$ even nonzero $t \in T$

In [ ]:
bridge_model.add_constraints(bridge_model.sum(x[(i,t)] for i in I) == 1 for t in T[1:] if t % 2 == 0)

Each person is at the start or end at the end of each time period

$s_{it} + f_{it} = 1, \quad \forall i \in i, \space t \in T$

In [ ]:
bridge_model.add_constraints(s[(i,t)] + f[(i,t)] == 1 for i in I for t in T)

The travel time at time $t \in T$ is the travel time of the slower walker

$w_{t} \ge c_{i}x_{it}, \quad \forall i \in I, \space t \in T$

In [ ]:
bridge_model.add_constraints(w[t] >= c[i]*x[(i,t)] for i in I for t in T[1:])


Logic for connecting x variables to f and p variables

$f_{p(t-1)} + x_{pt} = f_{pt}, \quad \forall$ odd $t \in T$

$s_{p(t-1)} + x_{pt} = s_{pt}, \quad \forall$ nonzero even $t \in T$

In [ ]:
bridge_model.add_constraints(f[(i,t-1)] + x[(i,t)] == f[(i,t)] for i in I for t in T if t % 2 == 1)
bridge_model.add_constraints(s[(i,t-1)] + x[(i,t)] == s[(i,t)] for i in I for t in T[1:] if t % 2 == 0)

# Solve the problem

In [ ]:
bridge_model.solve()

## Objective Function Value

In [ ]:
bridge_model.solution.objective_value

## Who crosses at each time

In [ ]:
x_values = bridge_model.solution.get_value_df(x)
x_values.loc[x_values.value == 1].rename(columns={"key_1":"Person","key_2":"Time Period"})[["Time Period","Person"]].sort_values('Time Period')

## Send Results to JSON

In [ ]:
bridge_model.solution.export("bridge_and_torch_solution.json")